In [0]:
customer_dim_data = [

(1,'manish','arwal','india','N','2022-09-15','2022-09-25'),
(2,'vikash','patna','india','Y','2023-08-12',None),
(3,'nikita','delhi','india','Y','2023-09-10',None),
(4,'rakesh','jaipur','india','Y','2023-06-10',None),
(5,'ayush','NY','USA','Y','2023-06-10',None),
(1,'manish','gurgaon','india','Y','2022-09-25',None),
]

customer_schema= ['id','name','city','country','active','effective_start_date','effective_end_date']

customer_dim_df = spark.createDataFrame(data= customer_dim_data,schema=customer_schema)

sales_data = [

(1,1,'manish','2023-01-16','gurgaon','india',380),
(77,1,'manish','2023-03-11','bangalore','india',300),
(12,3,'nikita','2023-09-20','delhi','india',127),
(54,4,'rakesh','2023-08-10','jaipur','india',321),
(65,5,'ayush','2023-09-07','mosco','russia',765),
(89,6,'rajat','2023-08-10','jaipur','india',321)
]

sales_schema = ['sales_id', 'customer_id','customer_name', 'sales_date', 'food_delivery_address','food_delivery_country', 'food_cost']

sales_df = spark.createDataFrame(data=sales_data,schema=sales_schema)

In [0]:
customer_dim_df.show()

+---+------+-------+-------+------+--------------------+------------------+
| id|  name|   city|country|active|effective_start_date|effective_end_date|
+---+------+-------+-------+------+--------------------+------------------+
|  1|manish|  arwal|  india|     N|          2022-09-15|        2022-09-25|
|  2|vikash|  patna|  india|     Y|          2023-08-12|              null|
|  3|nikita|  delhi|  india|     Y|          2023-09-10|              null|
|  4|rakesh| jaipur|  india|     Y|          2023-06-10|              null|
|  5| ayush|     NY|    USA|     Y|          2023-06-10|              null|
|  1|manish|gurgaon|  india|     Y|          2022-09-25|              null|
+---+------+-------+-------+------+--------------------+------------------+



In [0]:
sales_df.show()

+--------+-----------+-------------+----------+---------------------+---------------------+---------+
|sales_id|customer_id|customer_name|sales_date|food_delivery_address|food_delivery_country|food_cost|
+--------+-----------+-------------+----------+---------------------+---------------------+---------+
|       1|          1|       manish|2023-01-16|              gurgaon|                india|      380|
|      77|          1|       manish|2023-03-11|            bangalore|                india|      300|
|      12|          3|       nikita|2023-09-20|                delhi|                india|      127|
|      54|          4|       rakesh|2023-08-10|               jaipur|                india|      321|
|      65|          5|        ayush|2023-09-07|                mosco|               russia|      765|
|      89|          6|        rajat|2023-08-10|               jaipur|                india|      321|
+--------+-----------+-------------+----------+---------------------+-------------

In [0]:
joined_data=customer_dim_df.join(sales_df,sales_df['customer_id']==customer_dim_df['id'],'left')

In [0]:
display(joined_data)

id,name,city,country,active,effective_start_date,effective_end_date,sales_id,customer_id,customer_name,sales_date,food_delivery_address,food_delivery_country,food_cost
1,manish,arwal,india,N,2022-09-15,2022-09-25,77,1,manish,2023-03-11,bangalore,india,300
1,manish,arwal,india,N,2022-09-15,2022-09-25,1,1,manish,2023-01-16,gurgaon,india,380
2,vikash,patna,india,Y,2023-08-12,null,null,null,null,null,null,null,null
3,nikita,delhi,india,Y,2023-09-10,null,12,3,nikita,2023-09-20,delhi,india,127
4,rakesh,jaipur,india,Y,2023-06-10,null,54,4,rakesh,2023-08-10,jaipur,india,321
5,ayush,NY,USA,Y,2023-06-10,null,65,5,ayush,2023-09-07,mosco,russia,765
1,manish,gurgaon,india,Y,2022-09-25,null,77,1,manish,2023-03-11,bangalore,india,300
1,manish,gurgaon,india,Y,2022-09-25,null,1,1,manish,2023-01-16,gurgaon,india,380


In [0]:
df_new_records=joined_data.filter(((col('id')==col('customer_id')) & (col('food_delivery_address')!=col('city'))) & (col('active')=='Y'))\
    .withColumn('active',lit('Y'))\
    .withColumn('effective_start_date',col('sales_date'))\
    .withColumn('effective_end_date',lit(None)).select('id','name',col('food_delivery_address').alias('city'),'food_delivery_country','active','effective_start_date','effective_end_date')

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window

In [0]:
display(df_new_records)

id,name,city,food_delivery_country,active,effective_start_date,effective_end_date
1,manish,bangalore,india,Y,2023-03-11,null
5,ayush,mosco,russia,Y,2023-09-07,null


In [0]:
df_new_records=joined_data.filter(((col('id')==col('customer_id')) & (col('food_delivery_address')!=col('city'))) & (col('active')=='Y'))\
    .withColumn('active',lit('N'))

In [0]:
old_data=joined_data.filter(((col('id')==col('customer_id')) & (col('food_delivery_address')!=col('city'))) & (col('active')=='Y'))\
    .withColumn('active',lit('N'))\
    .withColumn('effective_end_date',col('sales_date')).select('id','name',col('city').alias('city'),'country','active','effective_start_date','effective_end_date')

In [0]:
old_data.show()

+---+------+-------+-------+------+--------------------+------------------+
| id|  name|   city|country|active|effective_start_date|effective_end_date|
+---+------+-------+-------+------+--------------------+------------------+
|  1|manish|gurgaon|  india|     N|          2022-09-25|        2023-03-11|
|  5| ayush|     NY|    USA|     N|          2023-06-10|        2023-09-07|
+---+------+-------+-------+------+--------------------+------------------+



In [0]:
new_customers=sales_df.join(customer_dim_df,sales_df['customer_id']==customer_dim_df['id'],'leftanti')\
                .withColumn('active',lit('Y'))\
                .withColumn('effective_end_date',lit(None))\
                .select(col('customer_id').alias('id'),
                        col('customer_name').alias('name'),
                        col('food_delivery_address').alias('city'),
                        col('food_delivery_country').alias('country'),
                        col('active'),
                        col('sales_date').alias('effective_start_date'),
                        col('effective_end_date')
                        )



In [0]:
df_new_records.show()

+---+------+-------+-------+------+--------------------+------------------+--------+-----------+-------------+----------+---------------------+---------------------+---------+
| id|  name|   city|country|active|effective_start_date|effective_end_date|sales_id|customer_id|customer_name|sales_date|food_delivery_address|food_delivery_country|food_cost|
+---+------+-------+-------+------+--------------------+------------------+--------+-----------+-------------+----------+---------------------+---------------------+---------+
|  1|manish|gurgaon|  india|     N|          2022-09-25|              null|      77|          1|       manish|2023-03-11|            bangalore|                india|      300|
|  5| ayush|     NY|    USA|     N|          2023-06-10|              null|      65|          5|        ayush|2023-09-07|                mosco|               russia|      765|
+---+------+-------+-------+------+--------------------+------------------+--------+-----------+-------------+----------

In [0]:
final_df=customer_dim_df.union(df_new_records).union(old_data).union(new_customers)

In [0]:
final_df.show()

+---+------+---------+-------+------+--------------------+------------------+
| id|  name|     city|country|active|effective_start_date|effective_end_date|
+---+------+---------+-------+------+--------------------+------------------+
|  1|manish|    arwal|  india|     N|          2022-09-15|        2022-09-25|
|  2|vikash|    patna|  india|     Y|          2023-08-12|              null|
|  3|nikita|    delhi|  india|     Y|          2023-09-10|              null|
|  4|rakesh|   jaipur|  india|     Y|          2023-06-10|              null|
|  5| ayush|       NY|    USA|     Y|          2023-06-10|              null|
|  1|manish|  gurgaon|  india|     Y|          2022-09-25|              null|
|  1|manish|bangalore|  india|     Y|          2023-03-11|              null|
|  5| ayush|    mosco| russia|     Y|          2023-09-07|              null|
|  1|manish|  gurgaon|  india|     N|          2022-09-25|        2023-03-11|
|  5| ayush|       NY|    USA|     N|          2023-06-10|      

In [0]:
final_df.show()

+---+------+---------+-------+------+--------------------+------------------+
| id|  name|     city|country|active|effective_start_date|effective_end_date|
+---+------+---------+-------+------+--------------------+------------------+
|  1|manish|    arwal|  india|     N|          2022-09-15|        2022-09-25|
|  2|vikash|    patna|  india|     Y|          2023-08-12|              null|
|  3|nikita|    delhi|  india|     Y|          2023-09-10|              null|
|  4|rakesh|   jaipur|  india|     Y|          2023-06-10|              null|
|  5| ayush|       NY|    USA|     Y|          2023-06-10|              null|
|  1|manish|  gurgaon|  india|     Y|          2022-09-25|              null|
|  1|manish|bangalore|  india|     Y|          2023-03-11|              null|
|  5| ayush|    mosco| russia|     Y|          2023-09-07|              null|
|  1|manish|  gurgaon|  india|     N|          2022-09-25|        2023-03-11|
|  5| ayush|       NY|    USA|     N|          2023-06-10|      

In [0]:
window=Window.partitionBy('id','active').orderBy(col('effective_start_date').desc())

In [0]:
final_df.withColumn('row_number',row_number().over(window)).filter(col('active')=='N').show()

+---+------+-------+-------+------+--------------------+------------------+----------+
| id|  name|   city|country|active|effective_start_date|effective_end_date|row_number|
+---+------+-------+-------+------+--------------------+------------------+----------+
|  1|manish|gurgaon|  india|     N|          2022-09-25|        2023-03-11|         1|
|  1|manish|  arwal|  india|     N|          2022-09-15|        2022-09-25|         2|
|  5| ayush|     NY|    USA|     N|          2023-06-10|        2023-09-07|         1|
+---+------+-------+-------+------+--------------------+------------------+----------+



In [0]:
final_df.show()

+---+------+---------+-------+------+--------------------+------------------+
| id|  name|     city|country|active|effective_start_date|effective_end_date|
+---+------+---------+-------+------+--------------------+------------------+
|  1|manish|    arwal|  india|     N|          2022-09-15|        2022-09-25|
|  2|vikash|    patna|  india|     Y|          2023-08-12|              null|
|  3|nikita|    delhi|  india|     Y|          2023-09-10|              null|
|  4|rakesh|   jaipur|  india|     Y|          2023-06-10|              null|
|  5| ayush|       NY|    USA|     Y|          2023-06-10|              null|
|  1|manish|  gurgaon|  india|     Y|          2022-09-25|              null|
|  1|manish|bangalore|  india|     Y|          2023-03-11|              null|
|  5| ayush|    mosco| russia|     Y|          2023-09-07|              null|
|  1|manish|  gurgaon|  india|     N|          2022-09-25|        2023-03-11|
|  5| ayush|       NY|    USA|     N|          2023-06-10|      

In [0]:
final_df.withColumn('row_number',row_number().over(window)).filter(~((col('active')=='Y') & (col('row_number')>1)) ).show()

+---+------+---------+-------+------+--------------------+------------------+----------+
| id|  name|     city|country|active|effective_start_date|effective_end_date|row_number|
+---+------+---------+-------+------+--------------------+------------------+----------+
|  1|manish|  gurgaon|  india|     N|          2022-09-25|        2023-03-11|         1|
|  1|manish|    arwal|  india|     N|          2022-09-15|        2022-09-25|         2|
|  1|manish|bangalore|  india|     Y|          2023-03-11|              null|         1|
|  2|vikash|    patna|  india|     Y|          2023-08-12|              null|         1|
|  3|nikita|    delhi|  india|     Y|          2023-09-10|              null|         1|
|  4|rakesh|   jaipur|  india|     Y|          2023-06-10|              null|         1|
|  5| ayush|       NY|    USA|     N|          2023-06-10|        2023-09-07|         1|
|  5| ayush|    mosco| russia|     Y|          2023-09-07|              null|         1|
|  6| rajat|   jaipur

In [0]:
final_df.withColumn('row_number',row_number().over(window)).filter(~((col('active')=='Y') & (col('row_number')>1)) ).show()

+---+------+---------+-------+------+--------------------+------------------+----------+
| id|  name|     city|country|active|effective_start_date|effective_end_date|row_number|
+---+------+---------+-------+------+--------------------+------------------+----------+
|  1|manish|  gurgaon|  india|     N|          2022-09-25|        2023-03-11|         1|
|  1|manish|    arwal|  india|     N|          2022-09-15|        2022-09-25|         2|
|  1|manish|bangalore|  india|     Y|          2023-03-11|              null|         1|
|  2|vikash|    patna|  india|     Y|          2023-08-12|              null|         1|
|  3|nikita|    delhi|  india|     Y|          2023-09-10|              null|         1|
|  4|rakesh|   jaipur|  india|     Y|          2023-06-10|              null|         1|
|  5| ayush|       NY|    USA|     N|          2023-06-10|        2023-09-07|         1|
|  5| ayush|    mosco| russia|     Y|          2023-09-07|              null|         1|
|  6| rajat|   jaipur